## Moneyball Exercise 15, The Iron Yard
- Creating the best team for the lowest cost.


* The key objective is to determine the 'best' players based on performance per salary.
* The current year data is being used to determine the performance.  
* The performance alogorthm will then be applied to each year. 


## Analysis plan:
1. read in only the full tables that will contain the variables of interest. 
2. the full tables will be trimmed to include those columns of interest.
3. a final table will be constructed using the combined data [data cleaned, columns transformed, stats calculated]

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
master = pd.read_csv('lahman-csv_2014-02-14/Master.csv', sep=',', header=0) 

In [12]:
master.head(3)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,205,75,R,R,2004-04-06,2013-09-28,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,...,Aaron,Tommie Lee,190,75,R,R,1962-04-10,1971-09-26,aarot101,aaronto01


In [44]:
mastertrim = master[['playerID','nameGiven','birthYear','finalGame']]

In [45]:
mastertrim.head()

,playerID,nameGiven,birthYear,finalGame
0,aardsda01,David Allan,1981,2013-09-28
1,aaronha01,Henry Louis,1934,1976-10-03
2,aaronto01,Tommie Lee,1939,1971-09-26
3,aasedo01,Donald William,1954,1990-10-03
4,abadan01,Fausto Andres,1972,2006-04-13


In [46]:
batting = pd.read_csv('lahman-csv_2014-02-14/Batting.csv', sep=',', header=0) 

In [47]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,aardsda01,2004,1,SFN,NL,11,11,0,0,0,...,0,0,0,0,0,0,0,0,0,11
1,aardsda01,2006,1,CHN,NL,45,43,2,0,0,...,0,0,0,0,0,0,1,0,0,45
2,aardsda01,2007,1,CHA,AL,25,2,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,aardsda01,2008,1,BOS,AL,47,5,1,0,0,...,0,0,0,1,0,0,0,0,0,5
4,aardsda01,2009,1,SEA,AL,73,3,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [48]:
battingtrim = batting[['playerID', 'yearID', 'teamID', 'AB', 'G','H','BB','HBP','SH']]

In [49]:
battingtrim.head()

,playerID,yearID,teamID,AB,G,H,BB,HBP,SH
0,aardsda01,2004,SFN,0,11,0,0,0,0
1,aardsda01,2006,CHN,2,45,0,0,0,1
2,aardsda01,2007,CHA,0,25,0,0,0,0
3,aardsda01,2008,BOS,1,47,0,0,0,0
4,aardsda01,2009,SEA,0,73,0,0,0,0


In [50]:
salaries = pd.read_csv('lahman-csv_2014-02-14/Salaries.csv', sep=',', header=0) 

In [51]:
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,BAL,AL,murraed02,1472819
1,1985,BAL,AL,lynnfr01,1090000
2,1985,BAL,AL,ripkeca01,800000
3,1985,BAL,AL,lacyle01,725000
4,1985,BAL,AL,flanami01,641667


In [52]:
salarytrim = salaries[['playerID', 'yearID','teamID','salary']]

In [53]:
salarytrim.head()

,playerID,yearID,teamID,salary
0,murraed02,1985,BAL,1472819
1,lynnfr01,1985,BAL,1090000
2,ripkeca01,1985,BAL,800000
3,lacyle01,1985,BAL,725000
4,flanami01,1985,BAL,641667


In [54]:
salarytrim.count()

playerID    23956
yearID      23956
teamID      23956
salary      23956
dtype: int64

In [55]:
mastertrim.count()

playerID     18354
nameGiven    18310
birthYear    18200
finalGame    18165
dtype: int64

In [56]:
battingtrim.count()

playerID    97889
yearID      97889
teamID      97889
AB          91476
G           97889
H           91476
BB          91476
HBP         88656
SH          85138
dtype: int64

Assume that the annual stats have been normalized across teams for any given player. 

In [57]:
bbfile = pd.merge(salarytrim, mastertrim)

In [58]:
bbfile.head(10)

,playerID,yearID,teamID,salary,nameGiven,birthYear,finalGame
0,murraed02,1985,BAL,1472819,Eddie Clarence,1956,1997-09-20
1,murraed02,1986,BAL,925814,Eddie Clarence,1956,1997-09-20
2,murraed02,1987,BAL,2044757,Eddie Clarence,1956,1997-09-20
3,murraed02,1988,BAL,2142904,Eddie Clarence,1956,1997-09-20
4,murraed02,1989,LAN,2331062,Eddie Clarence,1956,1997-09-20
5,murraed02,1990,LAN,2513703,Eddie Clarence,1956,1997-09-20
6,murraed02,1991,LAN,2628164,Eddie Clarence,1956,1997-09-20
7,murraed02,1992,NYN,4125000,Eddie Clarence,1956,1997-09-20
8,murraed02,1993,NYN,3375000,Eddie Clarence,1956,1997-09-20
9,murraed02,1994,CLE,3000000,Eddie Clarence,1956,1997-09-20


In [59]:
bbfile.count()

playerID     23956
yearID       23956
teamID       23956
salary       23956
nameGiven    23956
birthYear    23956
finalGame    23956
dtype: int64

In [60]:
totfile = pd.merge(bbfile,battingtrim)

In [61]:
totfile.head()

,playerID,yearID,teamID,salary,nameGiven,birthYear,finalGame,AB,G,H,BB,HBP,SH
0,murraed02,1985,BAL,1472819,Eddie Clarence,1956,1997-09-20,583,156,173,84,2,0
1,murraed02,1986,BAL,925814,Eddie Clarence,1956,1997-09-20,495,137,151,78,0,0
2,murraed02,1987,BAL,2044757,Eddie Clarence,1956,1997-09-20,618,160,171,73,0,0
3,murraed02,1988,BAL,2142904,Eddie Clarence,1956,1997-09-20,603,161,171,75,0,0
4,murraed02,1989,LAN,2331062,Eddie Clarence,1956,1997-09-20,594,160,147,87,2,0


In [62]:
totfile.count()


playerID     22871
yearID       22871
teamID       22871
salary       22871
nameGiven    22871
birthYear    22871
finalGame    22871
AB           20618
G            22871
H            20618
BB           20618
HBP          20609
SH           20618
dtype: int64

In [67]:
totfile['OBP'] = ((totfile['H'] + totfile['BB'] + totfile['HBP'])/(totfile['AB'] + totfile['BB'] + totfile['HBP'] + totfile['SH']))

In [68]:
totfile.head(3)

,playerID,yearID,teamID,salary,nameGiven,birthYear,finalGame,AB,G,H,BB,HBP,SH,OBP
0,murraed02,1985,BAL,1472819,Eddie Clarence,1956,1997-09-20,583,156,173,84,2,0,0.387145
1,murraed02,1986,BAL,925814,Eddie Clarence,1956,1997-09-20,495,137,151,78,0,0,0.399651
2,murraed02,1987,BAL,2044757,Eddie Clarence,1956,1997-09-20,618,160,171,73,0,0,0.353111


In [69]:
totfile.count()

playerID     22871
yearID       22871
teamID       22871
salary       22871
nameGiven    22871
birthYear    22871
finalGame    22871
AB           20618
G            22871
H            20618
BB           20618
HBP          20609
SH           20618
OBP          18108
dtype: int64

In [70]:
totfile.describe()

,yearID,salary,birthYear,AB,G,H,BB,HBP,SH,OBP
count,22871.000000,22871.000000,22871.000000,20618.000000,22871.00000,20618.000000,20618.000000,20609.000000,20618.000000,18108.000000
mean,1999.362162,1888500.460146,1969.843689,193.245805,67.87224,51.124794,19.085653,1.713766,1.887962,0.266165
std,8.020054,3100132.146168,8.812243,207.066756,47.76365,59.000818,24.602970,2.973277,3.008365,0.139909
min,1985.000000,0.000000,1939.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1993.000000,250000.000000,1964.000000,5.000000,29.00000,1.000000,0.000000,0.000000,0.000000,0.200000
50%,1999.000000,530000.000000,1970.000000,99.000000,56.00000,23.000000,8.000000,0.000000,0.000000,0.304445
75%,2006.000000,2150000.000000,1976.000000,366.000000,108.00000,96.000000,32.000000,2.000000,3.000000,0.345946
max,2013.000000,33000000.000000,1993.000000,716.000000,163.00000,262.000000,232.000000,35.000000,39.000000,1.000000


Need to add the Appearance data to capture the key 9 positions associated with players:

In [71]:
positions = pd.read_csv('lahman-csv_2014-02-14/Appearances.csv', sep=',', header=0) 

In [72]:
positions.head()

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,NaN,1,1,0,0,...,0,0,1,0,0,0,0,NaN,NaN,NaN
1,1871,RC1,NaN,addybo01,25,NaN,25,25,0,0,...,22,0,3,0,0,0,0,NaN,NaN,NaN
2,1871,CL1,NaN,allisar01,29,NaN,29,29,0,0,...,2,0,0,0,29,0,29,NaN,NaN,NaN
3,1871,WS3,NaN,allisdo01,27,NaN,27,27,0,27,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1871,RC1,NaN,ansonca01,25,NaN,25,25,0,5,...,2,20,0,1,0,0,1,NaN,NaN,NaN


In [73]:
positiontrim = positions[['playerID','yearID','teamID','G_p', 'G_c','G_1b','G_2b','G_3b','G_ss','G_lf','G_cf','G_of']]

In [74]:
positiontrim.head()

,playerID,yearID,teamID,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of
0,abercda01,1871,TRO,0,0,0,0,0,1,0,0,0
1,addybo01,1871,RC1,0,0,0,22,0,3,0,0,0
2,allisar01,1871,CL1,0,0,0,2,0,0,0,29,29
3,allisdo01,1871,WS3,0,27,0,0,0,0,0,0,0
4,ansonca01,1871,RC1,0,5,1,2,20,0,1,0,1


In [75]:
finalbb = pd.merge(totfile, positiontrim)

In [76]:
finalbb.head()

,playerID,yearID,teamID,salary,nameGiven,birthYear,finalGame,AB,G,H,...,OBP,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of
0,murraed02,1985,BAL,1472819,Eddie Clarence,1956,1997-09-20,583,156,173,...,0.387145,0,0,154,0,0,0,0,0,0
1,murraed02,1986,BAL,925814,Eddie Clarence,1956,1997-09-20,495,137,151,...,0.399651,0,0,119,0,0,0,0,0,0
2,murraed02,1987,BAL,2044757,Eddie Clarence,1956,1997-09-20,618,160,171,...,0.353111,0,0,156,0,0,0,0,0,0
3,murraed02,1988,BAL,2142904,Eddie Clarence,1956,1997-09-20,603,161,171,...,0.362832,0,0,103,0,0,0,0,0,0
4,murraed02,1989,LAN,2331062,Eddie Clarence,1956,1997-09-20,594,160,147,...,0.345534,0,0,159,0,2,0,0,0,0


In [77]:
finalbb.count()

playerID     22788
yearID       22788
teamID       22788
salary       22788
nameGiven    22788
birthYear    22788
finalGame    22788
AB           20537
G            22788
H            20537
BB           20537
HBP          20528
SH           20537
OBP          18048
G_p          22788
G_c          22788
G_1b         22788
G_2b         22788
G_3b         22788
G_ss         22788
G_lf         22788
G_cf         22788
G_of         22788
dtype: int64

In [113]:
decision_set= finalbb[['playerID','yearID','teamID','nameGiven', 'G_p','G_c','G_1b','G_2b','G_3b','G_ss','G_lf','G_cf','G_of','salary', 'OBP' ]]

In [114]:
decision_set.head()

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
0,murraed02,1985,BAL,Eddie Clarence,0,0,154,0,0,0,0,0,0,1472819,0.387145
1,murraed02,1986,BAL,Eddie Clarence,0,0,119,0,0,0,0,0,0,925814,0.399651
2,murraed02,1987,BAL,Eddie Clarence,0,0,156,0,0,0,0,0,0,2044757,0.353111
3,murraed02,1988,BAL,Eddie Clarence,0,0,103,0,0,0,0,0,0,2142904,0.362832
4,murraed02,1989,LAN,Eddie Clarence,0,0,159,0,2,0,0,0,0,2331062,0.345534


OBP * 100000 / Salary is the weighed decision score (DS).  For any given OBP, the DS will go up as the salary goes down.  

In [115]:
decision_set.sort('salary', ascending=True)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
4986,jamesdi01,1993,NYA,Dion,0,0,1,0,0,0,91,14,103,0,0.389920
9937,silveda01,1993,NYA,David Joseph,0,0,0,0,3,4,0,0,0,10900,0.423077
4137,riverlu01,1986,MON,Luis Antonio,0,0,0,0,0,55,0,0,0,60000,0.284946
313,slatoji01,1986,DET,James Michael,22,0,0,0,0,0,0,0,0,60000,NaN
3060,whitede03,1986,CAL,Devon Markes,0,0,0,0,0,0,18,7,28,60000,0.315789
4382,bondsba01,1986,PIT,Barry Lamar,0,0,0,0,0,0,0,110,110,60000,0.329876
3676,brantmi01,1986,SEA,Michael Charles,0,0,0,0,0,0,1,25,25,60000,0.265487
3043,finlech01,1986,CAL,Charles Edward,25,0,0,0,0,0,0,0,0,60000,NaN
2145,duncama01,1985,LAN,Mariano,0,0,0,19,0,123,0,0,0,60000,0.288961
124,sheetla01,1985,BAL,Larry Kent,0,0,1,0,0,0,1,0,9,60000,0.323120


Cleaning the data:
Salary Range = 0 to millions.   The zero value is an outlier.  
OBP range is NaN to 1.          The 1 represents an unlikely perfect score.  1 is an outlier. 
Year range = 2013 to 1985       

In [116]:
decision_set = decision_set[decision_set.salary != 0]

In [119]:
decision_set.sort('salary', ascending=True)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
9937,silveda01,1993,NYA,David Joseph,0,0,0,0,3,4,0,0,0,10900,0.423077
3582,mcgwima01,1986,OAK,Mark David,0,0,0,0,16,0,0,0,0,60000,0.258621
4050,hamilje01,1986,LAN,Jeffrey Robert,0,0,0,0,67,1,0,0,0,60000,0.234899
124,sheetla01,1985,BAL,Larry Kent,0,0,1,0,0,0,1,0,9,60000,0.323120
2451,anderla02,1986,HOU,Larry Eugene,38,0,0,0,0,0,0,0,0,60000,0.000000
4023,mizerjo01,1986,HOU,John Joseph,0,42,0,0,0,0,0,0,0,60000,0.377358
2499,kniceal01,1985,PHI,Alan Lee,0,0,1,0,0,0,0,0,0,60000,0.000000
2538,mazzile01,1986,NYN,Lee Louis,0,0,8,0,0,0,8,0,10,60000,0.416667
2418,carltst01,1986,CHA,Steven Norman,10,0,0,0,0,0,0,0,0,60000,NaN
530,ruhleve01,1986,CAL,Vernon Gerald,16,0,0,0,0,0,0,0,0,60000,NaN


In [120]:
decision_set.sort('OBP', ascending=False)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
6480,valdese01,1989,ATL,Sergio Sanchez,19,0,0,0,0,0,0,0,0,68000,1
13492,baileco01,1997,SFN,Philip Cory,7,0,0,0,0,0,0,0,0,181000,1
14480,howrybo01,2006,CHN,Bobby Dean,84,0,0,0,0,0,0,0,0,3000000,1
14495,wardbr01,1999,CHA,Bryan Matthew,40,0,0,0,0,0,0,0,0,215000,1
245,clemero02,1996,BOS,William Roger,34,0,0,0,0,0,0,0,0,5500000,1
13587,castica02,1997,CHA,Carlos,37,0,0,0,0,0,0,0,0,150000,1
18875,kimsu01,2006,COL,Sun-Woo,6,0,0,0,0,0,0,0,0,600000,1
21754,hernada01,2012,ARI,David Jose,72,0,0,0,0,0,0,0,0,499000,1
15722,duvalmi01,1999,TBA,Michael Alan,40,0,0,0,0,0,0,0,0,215000,1
10188,borbope02,1996,ATL,Pedro Felix,43,0,0,0,0,0,0,0,0,142500,1


In [121]:
decision_set = decision_set[decision_set.OBP != 1]

In [122]:
decision_set.sort('OBP', ascending=False)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
7182,mantoje01,2000,COL,Jeffrey Paul,0,0,1,0,1,0,0,0,0,450000,0.857143
10227,kleskry01,2006,SDN,Ryan Anthony,0,0,0,0,0,0,0,0,0,9000000,0.833333
15217,aybarma01,2001,CHN,Manuel Antonio,17,0,0,0,0,0,0,0,0,665000,0.750000
10195,jonesch06,1993,ATL,Larry Wayne,0,0,0,0,0,3,0,0,0,109000,0.750000
6372,rogerke01,2002,TEX,Kenneth Scott,33,0,0,0,0,0,0,0,0,7500000,0.750000
15061,woodabr01,1999,CHN,David Bradley,6,0,0,0,0,0,0,0,0,200000,0.750000
17128,stechge01,2001,SLN,Gene Urban,67,0,0,0,0,0,0,0,0,205000,0.750000
8887,munozmi01,1995,COL,Michael Anthony,64,0,0,0,0,0,0,0,0,300000,0.750000
17649,tucketj01,2002,MON,Thomas John,57,0,0,0,0,0,0,0,0,200000,0.750000
251,clemero02,2002,NYA,William Roger,29,0,0,0,0,0,0,0,0,10300000,0.666667


In [123]:
decision_set.sort('OBP', ascending=True)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
11549,thompma01,2000,SLN,Mark Radford,20,0,0,0,0,0,0,0,0,400000,0
15586,lincomi01,2002,PIT,Michael George,55,0,0,0,0,0,0,0,0,267500,0
15585,lincomi01,1999,MIN,Michael George,18,0,0,0,0,0,0,0,0,200000,0
15535,reichda01,2001,KCA,Daniel Robert,27,0,0,0,0,0,0,0,0,265000,0
15534,reichda01,2000,KCA,Daniel Robert,44,0,0,0,0,0,0,0,0,205000,0
15532,morenor01,2004,NYN,Orber,33,0,0,0,0,0,0,0,0,317500,0
15525,fussech01,2000,KCA,Christopher Wren,20,0,0,0,0,0,0,0,0,210000,0
15516,byrdati01,2010,HOU,Timothy Christopher,64,0,0,0,0,0,0,0,0,1600000,0
15515,byrdati01,2009,HOU,Timothy Christopher,76,0,0,0,0,0,0,0,0,1000000,0
15490,witasja01,2004,SDN,Gerald Alphonse,44,0,0,0,0,0,0,0,0,1750000,0


In [124]:
decision_set = decision_set[decision_set.OBP != 0]

In [125]:
decision_set.sort('OBP', ascending=True)[:10]

,playerID,yearID,teamID,nameGiven,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
17609,ishiika01,2003,LAN,Kazuhisa,27,0,0,0,0,0,0,0,0,2575000,0.023256
4224,carmado01,1989,PHI,Donald Wayne,49,0,0,0,0,0,0,0,0,575000,0.024390
18769,haranaa01,2005,CIN,Aaron Michael,32,0,0,0,0,0,0,0,0,440000,0.025641
16186,bensokr01,2003,PIT,Kristen James,18,0,0,0,0,0,0,0,0,4300000,0.027027
17562,beckejo02,2002,FLO,Joshua Patrick,23,0,0,0,0,0,0,0,0,2156250,0.027778
16812,davisdo02,2004,MIL,Douglas,34,0,0,0,0,0,0,0,0,450000,0.028169
16374,redmama01,2003,FLO,Mark Allen,29,0,0,0,0,0,0,0,0,2150000,0.030303
11075,worreti01,1993,SDN,Timothy Howard,21,0,0,0,0,0,0,0,0,109000,0.030303
18282,dayza01,2004,MON,Stephen Zachary,19,0,0,0,0,0,0,0,0,315000,0.031250
4456,lacosmi01,1990,SFN,Michael James,13,0,0,0,0,0,0,0,0,1150000,0.032258


In [126]:
decision_set.describe()

,yearID,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP
count,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,15787.000000
mean,1999.067131,13.430263,6.421347,6.647878,6.382569,6.492912,6.369270,6.712671,6.381059,19.124909,1900089.323233,0.297002
std,8.043047,20.546283,23.374884,25.232819,24.681537,24.511208,25.698181,22.565072,24.650516,41.215426,3116771.237894,0.088776
min,1985.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10900.000000,0.023256
25%,1993.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,250000.000000,0.256579
50%,1999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,550000.000000,0.314917
75%,2006.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2170833.500000,0.350420
max,2013.000000,94.000000,151.000000,162.000000,162.000000,162.000000,163.000000,162.000000,162.000000,162.000000,33000000.000000,0.857143


In [127]:
decision_set.count()

playerID     20527
yearID       20527
teamID       20527
nameGiven    20527
G_p          20527
G_c          20527
G_1b         20527
G_2b         20527
G_3b         20527
G_ss         20527
G_lf         20527
G_cf         20527
G_of         20527
salary       20527
OBP          15787
dtype: int64

In [128]:
decision_set['DS'] = (decision_set['OBP'] * 100000) / decision_set['salary']

In [129]:
decision_set.describe()

,yearID,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_of,salary,OBP,DS
count,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,20527.000000,15787.000000,15787.000000
mean,1999.067131,13.430263,6.421347,6.647878,6.382569,6.492912,6.369270,6.712671,6.381059,19.124909,1900089.323233,0.297002,0.079486
std,8.043047,20.546283,23.374884,25.232819,24.681537,24.511208,25.698181,22.565072,24.650516,41.215426,3116771.237894,0.088776,0.102226
min,1985.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10900.000000,0.023256,0.000250
25%,1993.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,250000.000000,0.256579,0.011740
50%,1999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,550000.000000,0.314917,0.044225
75%,2006.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2170833.500000,0.350420,0.104040
max,2013.000000,94.000000,151.000000,162.000000,162.000000,162.000000,163.000000,162.000000,162.000000,162.000000,33000000.000000,0.857143,3.881440
